<a href="https://colab.research.google.com/github/raima2000/Pharmacovigilance_detection/blob/main/bert_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel("/content/psyTAR_selected.xlsx")
df.head()

,comment,ADR_Status
0,I am detoxing from Lexapro now. I slowly cut m...,ADR
1,Just TWO tablets of Lexapro 10mg completely de...,ADR
2,Be careful with this medication. This was my ...,ADR
3,"It didn't help me out, at all. My anxiety is w...",ADR
4,I would not suggest taking this medication. I ...,ADR


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming df1 is your DataFrame with 'comment' and 'ADR_Status' columns

# Encode labels
label_encoder = LabelEncoder()
df['ADR_Status_encoded'] = label_encoder.fit_transform(df['ADR_Status'])

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Assuming you've already loaded and prepared your dataset in a DataFrame called `df`.

# Split your data into training and testing sets
X = df['comment'].tolist()
y = df['ADR_Status_encoded'].tolist()  # Ensure labels are in integer format
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and prepare data
max_length = 128

def tokenize_text(text):
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors='tf'
    )
    return inputs

# Tokenize the training and testing data
X_train_encoded = [tokenize_text(text) for text in X_train]
X_test_encoded = [tokenize_text(text) for text in X_test]

# Pad the sequences to have consistent lengths
X_train_input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_dict['input_ids'][0].numpy() for input_dict in X_train_encoded], maxlen=max_length, padding='post')
X_test_input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_dict['input_ids'][0].numpy() for input_dict in X_test_encoded], maxlen=max_length, padding='post')
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Define training arguments
train_args = tf.keras.optimizers.AdamW(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Create a model
model.compile(optimizer=train_args, loss=loss_fn, metrics=['accuracy'])

# Train the model
model.fit(
    X_train_input_ids,
    y_train,
    epochs=3,
    batch_size=32,
    validation_split=0.2
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_input_ids, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model
model.save('fine_tuned_model')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
16/16 [==============================] - 906s 53s/step - loss: 0.3203 - accuracy: 0.9124 - val_loss: 0.2201 - val_accuracy: 0.9512
Epoch 2/3
16/16 [==============================] - 845s 53s/step - loss: 0.1998 - accuracy: 0.9084 - val_loss: 0.1801 - val_accuracy: 0.9106
Epoch 3/3
5/5 [==============================] - 68s 13s/step - loss: 0.1697 - accuracy: 0.9351
Test Loss: 0.16965577006340027, Test Accuracy: 0.9350649118423462


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test_input_ids)
y_pred_classes = np.argmax(y_pred.logits, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

5/5 [==============================] - 82s 13s/step
Precision: 0.5714285714285714
Recall: 0.36363636363636365
F1 Score: 0.4444444444444444


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# # Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[140   3]
 [  7   4]]
